<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/09_2_langgraph_Self_RAG.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Upstage Self-RAG

Self-RAG is a strategy for RAG that incorperates [self-reflection / self-grading on retrieved documents and generations](https://blog.langchain.dev/agentic-rag-with-langgraph/). It leverages LLM to make decisions in the answer generation steps, hence improves quality of the final answer.

See [Self-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection](https://arxiv.org/abs/2310.11511) paper for more detailed information.

In this example, we will implement some of these ideas. With [Upstage integrations](https://python.langchain.com/docs/integrations/providers/upstage/) and [LangGraph](https://python.langchain.com/docs/langgraph), you can achieve this with just a few additional lines of code. Particularly, it uses Upstage Solar Mini chat model, Embeddings model, Layout Analysis for document understanding and retrieval, and Groundedness Check for verifying the generative model's response.

![image.png](attachment:image.png)

## Environment

In [ ]:
!pip install -qU langchain-core langchain-upstage langchain-chroma langchain langgraph
!pip install -qU python-dotenv

### Environment variables

Set up environment variables
* UPSTAGE_API_KEY

Optionally, set these environment variables to use [LangSmith](https://docs.smith.langchain.com/) for tracing (shown at the bottom)

```
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
LANGCHAIN_API_KEY=YOUR_KEY
```

In [2]:
# @title set API key
from pprint import pprint
import os
import getpass

import warnings

warnings.filterwarnings("ignore")

UPSTAGE_API_KEY = getpass.getpass("Enter your API Key")
_ = os.environ.setdefault("UPSTAGE_API_KEY", UPSTAGE_API_KEY)

LANGCHAIN_API_KEY = getpass.getpass("Enter your API Key")
_ = os.environ.setdefault("LANGCHAIN_API_KEY", LANGCHAIN_API_KEY)

Enter your API Key··········
Enter your API Key··········


In [3]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

upstage_api_key_env_name = "UPSTAGE_API_KEY"
langchain_api_key_env_name = "LANGCHAIN_API_KEY"


def load_env():
    if "google.colab" in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata

        upstage_api_key = userdata.get(upstage_api_key_env_name)
        langchain_api_key = userdata.get(langchain_api_key_env_name)
        return os.environ.setdefault(
            "UPSTAGE_API_KEY", upstage_api_key
        ), os.environ.setdefault("LANGCHAIN_API_KEY", langchain_api_key)
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv

        load_dotenv()
        return os.environ.get(upstage_api_key_env_name), os.environ.get(
            langchain_api_key_env_name
        )


UPSTAGE_API_KEY, LANGCHAIN_API_KEY = load_env()

## Retriever

### Prepare a file

This example will be using `docs/Upstage_Solar_DUS.pdf`. You can optionally add more files to the `docs` directory.

### Layout analysis

Prepare a function to use the Upstage [UpstageDocumentParseLoader](https://python.langchain.com/docs/integrations/document_loaders/upstage/) for document processing.

In [11]:
from typing import List
from langchain_core.documents import Document
from langchain_upstage import UpstageDocumentParseLoader


def document_parser(filenames: str) -> List[Document]:
    document_parser_loader = UpstageDocumentParseLoader(filenames, split="element")
    return document_parser_loader.load()


# Add more files if you'd like to.
filenames = [
    "pdfs/Upstage_Solar_DUS.pdf",
]

docs = document_parser(filenames)
print(f"number of documents: {len(docs)}")
print(docs[0])

number of documents: 188
page_content='<h1 id='0' style='font-size:20px'>SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective<br>Depth Up-Scaling</h1>' metadata={'id': 0, 'page': 1, 'category': 'heading1', 'coordinates': [{'x': 0.1194, 'y': 0.0808}, {'x': 0.8793, 'y': 0.0808}, {'x': 0.8793, 'y': 0.1197}, {'x': 0.1194, 'y': 0.1197}]}


In [20]:
print(type(docs[0]))

<class 'langchain_core.documents.base.Document'>


In [44]:
docs[170]

Document(metadata={'page': 11, 'id': 10, 'bounding_box': '[{"x": 634, "y": 462}, {"x": 882, "y": 462}, {"x": 882, "y": 490}, {"x": 634, "y": 490}]', 'category': 'paragraph'}, page_content="<p id='10' data-category='paragraph' style='font-size:14px'>B.2 Mixture of Experts</p>")

### Indexing

Let's index the file.

In [45]:
from langchain_chroma import Chroma
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_core.vectorstores import VectorStore

#Converts complex metadata values into JSON strings
def convert_complex_metadata(metadata):
    for key, value in metadata.items():
        if isinstance(value, list):
            metadata[key] = str(value)
    return metadata

# Process complex metadata in the list of documents
for doc in docs:
    doc.metadata = convert_complex_metadata(doc.metadata)


db: VectorStore = Chroma(
    embedding_function=UpstageEmbeddings(model="solar-embedding-1-large")
)


retriever = db.as_retriever()

db.add_documents(docs)

['b0b69a07-b679-42fc-9b80-dc5ae2474c37',
 '1e16829b-9890-4041-b20e-38cbdebbdd01',
 '1b37e95e-cfc7-4f12-bd8e-3762e7593c0d',
 'a5966b7c-c1a9-404a-b311-6e78b5a5adc8',
 '6d58fe64-30f5-4c7c-bd0b-321d6daceae1',
 'c68a10f6-76cd-4045-a7c6-5ac02a772ab6',
 '7039ef7f-ab52-4882-b556-f626f9b33521',
 '6bbb762a-5ee7-463e-bda1-b1fecfa48e4c',
 '0a3bb34a-7df4-4964-9577-7032c40f4eb6',
 'e98ffe2e-0e6f-468e-8cd8-adcb80ad24d8',
 'bca8e6f3-d37e-4189-a116-ab8c222037ff',
 '62857ac3-2d25-41c8-b152-891810a43d32',
 '3bcdfb66-ea08-4f26-83c0-5a5abf828174',
 '86200454-09c3-46cd-81fa-883a13498304',
 'a9380c3b-e38d-4811-bd6b-2f38cd008b5e',
 '2ac750a7-98c3-4725-a91c-a2b315d25369',
 'd987bce0-80c5-409a-8c13-6ca6c187101d',
 '082047bb-aa85-40f6-8c69-4fa187f1ac3c',
 '33449e67-4357-40a8-b2e2-98bf3dac43e8',
 '3c030f5a-f56e-4301-ad23-9aaa9c8242db',
 'b1915d55-438e-4183-ae7d-b6915c3a6ada',
 '65687031-672e-40ea-b5fa-fac72b51f345',
 '1a23061a-cb4a-4f01-b201-1f15633baf48',
 'f46114eb-0a96-4ae9-b891-a96ca89a729f',
 '0eb49e55-3360-

## Graph State
Let's first define the LangGraph state.

In [46]:
from typing import TypedDict


class RagState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        context: retrieved context
        question: question asked by the user
        answer: generated answer to the question
        groundedness: groundedness of the assistant's response
    """

    context: str
    question: str
    answer: str
    groundedness: str

## RAG

Let's prepare RAG pipeline. It uses the Upstage [Solar chat model](https://python.langchain.com/docs/integrations/chat/upstage/).

In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate


template = """Answer the question based only on the given context.
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatUpstage(model="solar-pro")

# Solar model answer generation, given the context and question
model_chain = prompt | model | StrOutputParser()


def format_documents(docs: List[Document]) -> str:
    return "\n".join([doc.page_content for doc in docs])


def retrieve(state: RagState) -> RagState:
    docs = retriever.invoke(state["question"])
    context = format_documents(docs)
    return RagState(context=context)


def model_answer(state: RagState) -> RagState:
    response = model_chain.invoke(state)
    return RagState(answer=response)

Now, let's prepare a logic for using Upstage [Groundedness Check](https://python.langchain.com/docs/integrations/tools/upstage_groundedness_check/).

In [48]:
from langchain_upstage import GroundednessCheck

gc = GroundednessCheck()


def groundedness_check(state: RagState) -> RagState:
    response = gc.run({"context": state["context"], "answer": state["answer"]})
    return RagState(groundedness=response)


def groundedness_condition(state: RagState) -> RagState:
    return state["groundedness"]

## Build Graph

Finally, let's put everything together and build the graph! The graph looks like the following.

![image.png](attachment:image.png)

In [49]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(RagState)
workflow.add_node("retrieve", retrieve)
workflow.add_node("model", model_answer)
workflow.add_node("groundedness_check", groundedness_check)

workflow.add_edge("retrieve", "model")
workflow.add_edge("model", "groundedness_check")
workflow.add_conditional_edges(
    "groundedness_check",
    groundedness_condition,
    {
        "grounded": END,
        "notGrounded": "model",
        "notSure": "model",
    },
)
workflow.set_entry_point("retrieve")

app = workflow.compile()

## Running the graph

Let's now test the graph.

In [50]:
inputs = {"question": "What is Solar?"}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':{value}")
    print("\n---\n")

Node 'retrieve':{'context': "<br><p id='94' data-category='paragraph' style='font-size:16px'>Additionally, we ensure that SOLAR complies<br>with general ethical considerations in all aspects<br>of its operation. This includes adherence to pri-<br>vacy norms, respect for intellectual property, and<br>ensuring the absence of bias in our algorithms. Our<br>commitment to these ethical principles is unwaver-<br>ing, and we believe it significantly contributes to<br>the credibility and societal acceptance of SOLAR.</p>\n<br><p id='94' data-category='paragraph' style='font-size:16px'>Additionally, we ensure that SOLAR complies<br>with general ethical considerations in all aspects<br>of its operation. This includes adherence to pri-<br>vacy norms, respect for intellectual property, and<br>ensuring the absence of bias in our algorithms. Our<br>commitment to these ethical principles is unwaver-<br>ing, and we believe it significantly contributes to<br>the credibility and societal acceptance of S

LangSmith Traces -

* https://smith.langchain.com/public/5ce3f275-b93b-48d1-a718-88139ae2e00b/r